In [1]:
import time
from queue import Empty as QueueEmpty
import heapq

In [ ]:
import time
from queue import Empty as QueueEmpty
import heapq

def transactionEngine(stockId, queue, dbQueue, iTQueue, logQueue, users, shutdownEvent):
    def marketTransaction(request):
        # Returns a list of internal transactions/ db transactions and user tranasactions
        nonlocal transactions, stockId
        timeStamp = time.time()
        side = request.get("side")
        internalTransactions = []   # Transactions from one user to another
        dbTransactions = [] # Database Transactions
        userTransactions = []   # User Transactions
        if side == "buy":
            buyerTransaction = request
            numberOfStocksRequired = buyerTransaction.get("quantity")
            totalStockBrought = 0; amountPaid = numberOfStocksRequired * buyerTransaction.get("pricePerUnit")
            buyerId = buyerTransaction.get("uId"); buyerTransactionId = buyerTransaction.get("tId")
            priceToBePaidByBuyer = 0
            
            while numberOfStocksRequired > 0 and len(transactions["sell"]) > 0:
                priceOfBestSell, thisTimeStamp, bestSell = heapq.heappop(transactions["sell"]) # 
                numberOfStocksAvailableInBestSell = bestSell.get("quantity")

                stocksInThisTransaction = min(numberOfStocksAvailableInBestSell, numberOfStocksRequired)
                numberOfStocksAvailableInBestSell -= stocksInThisTransaction
                numberOfStocksRequired -= stocksInThisTransaction
                totalStockBrought += stocksInThisTransaction

                priceInThisTransaction = stocksInThisTransaction * priceOfBestSell
                priceToBePaidByBuyer += priceInThisTransaction

                transactions["marketPrice"] = priceOfBestSell

                sellerId = bestSell.get("uId")
                sellerTid = bestSell.get("tId")
                internalTransactionRequest = {
                    "stockId": stockId,
                    "sellerId": sellerId,
                    "sellerTid": sellerTid,
                    "buyerId": buyerId,
                    "buyerTid": buyerTransactionId,
                    "noOfStocks": stocksInThisTransaction,
                    "amount": priceInThisTransaction,
                    "timeStamp": timeStamp
                }

                dbTransactionRequest = {
                    "tId": sellerTid,
                    "uId": sellerId,
                    "stockId": stockId,
                    "side": "sell",
                    "orderType": "market",
                    "quantity": stocksInThisTransaction,
                    "pricePerUnit": priceOfBestSell,
                    "status": "PARTIAL" if numberOfStocksAvailableInBestSell > 0 else "COMPLETED",
                    "timeStamp": timeStamp
                }

                userTransactionRequest = [
                        {
                            "action": "add",
                            "resource": "money",
                            "uId": sellerId,
                            "quantity": priceInThisTransaction
                        }
                    ]

                internalTransactions.append(internalTransactionRequest)
                dbTransactions.append(dbTransactionRequest)
                userTransactions.append(userTransactionRequest)

                if numberOfStocksAvailableInBestSell > 0:
                    # If there are more stocks to sell than the required, add them back to the heap
                    bestSell["quantity"] = numberOfStocksAvailableInBestSell
                    heapq.heappush(transactions["sell"], (priceOfBestSell, thisTimeStamp, bestSell))
            
            # If no stocks were brought then add the empty request
            if totalStockBrought == 0:
                dbTransactionRequest = {
                    "tId": buyerTransactionId,
                    "uId": buyerId,
                    "stockId": stockId,
                    "side": "buy",
                    "orderType": "market",
                    "quantity": totalStockBrought,
                    "pricePerUnit": 0,
                    "status": "IN-COMPLETE",
                    "timeStamp": timeStamp
                }

                userTransactionRequest = {
                    "action": "add",
                    "resource": "money",
                    "uId": buyerId,
                    "quantity": amountPaid
                }
                return [], [dbTransactionRequest], [userTransactionRequest]
            
            # Now either there are no more stocks left to buy or the request is satisfied
            internalTransactionRequest = {
                "stockId": stockId,
                "buyerId": buyerId,
                "buyerTid": buyerTransactionId,
                "noOfStocks": totalStockBrought,
                "amount": priceToBePaidByBuyer,
                "timeStamp": timeStamp
            }

            dbTransactionRequest = {
                "tId": buyerTransactionId,
                "uId": buyerId,
                "stockId": stockId,
                "side": "buy",
                "orderType": "market",
                "quantity": totalStockBrought,
                "pricePerUnit": priceToBePaidByBuyer / totalStockBrought if totalStockBrought else 0,
                "status": "PARTIAL" if numberOfStocksRequired > 0 else "COMPLETED",
                "timeStamp": timeStamp
            }

            userTransactionRequest = [
                {
                    "action": "add",
                    "resource": "stock",
                    "uId": buyerId,
                    "stockId": stockId,
                    "quantity": totalStockBrought
                }
            ]

            internalTransactions.append(internalTransactionRequest)
            dbTransactions.append(dbTransactionRequest)
            userTransactions.append(userTransactionRequest)

            return internalTransactions, dbTransactions, userTransactions

        else:
            # This is Sell Part
            sellerTransaction = request
            sellerId, sellerTransactionId = sellerTransaction.get("uId"), sellerTransaction.get("tId")
            totalStockSold = 0; totalAmountRecieved = 0
            numberOfStockAvailable = sellerTransaction.get("quantity")
            while numberOfStockAvailable > 0 and len(transactions["buy"]) > 0: # 
                buyPrice, thisTimeStamp, bestBuy = heapq.heappop(transactions["buy"])
                buyPrice *= -1  # Since the data is saved in negative prices for max-heap
                numberOfBestStocksToBuy = bestBuy.get("quantity")

                stocksSoldInThisTransaction = min(numberOfStockAvailable, numberOfBestStocksToBuy)
                priceInThisTransaction = buyPrice * stocksSoldInThisTransaction
                totalAmountRecieved += priceInThisTransaction

                transactions["marketPrice"] = buyPrice
                numberOfStockAvailable -= stocksSoldInThisTransaction
                numberOfBestStocksToBuy -= stocksSoldInThisTransaction
                totalStockSold += stocksSoldInThisTransaction

                buyerId = bestBuy.get("uId"); buyerTid = bestBuy.get("tId")
                internalTransactionRequest = {
                    "stockId": stockId,
                    "sellerId": sellerId,
                    "sellerTid": sellerTransactionId,
                    "buyerId": buyerId,
                    "buyerTid": buyerTid,
                    "noOfStocks": stocksSoldInThisTransaction,
                    "amount": priceInThisTransaction,
                    "timeStamp": timeStamp
                }

                dbTransactionRequest = {
                    "tId": buyerTid,
                    "uId": buyerId,
                    "stockId": stockId,
                    "side": "buy",
                    "orderType": "market",
                    "quantity": stocksSoldInThisTransaction,
                    "pricePerUnit": buyPrice,
                    "status": "PARTIAL" if numberOfBestStocksToBuy > 0 else "COMPLETE",
                    "timeStamp": timeStamp
                }

                userTransactionRequest = [
                {
                    "action": "add",
                    "resource": "stock",
                    "uId": buyerId,
                    "stockId": stockId,
                    "quantity": stocksSoldInThisTransaction
                }
                ]

                if numberOfBestStocksToBuy > 0:
                    bestBuy["quantity"] = numberOfBestStocksToBuy
                    heapq.heappush(transactions["buy"], (-1 * buyPrice, thisTimeStamp, bestBuy))
            
            if totalStockSold == 0:
                dbTransactionRequest = {
                    "tId": sellerTransactionId,
                    "uId": sellerId,
                    "stockId": stockId,
                    "side": "sell",
                    "orderType": "market",
                    "quantity": totalStockSold,
                    "pricePerUnit": 0,
                    "status": "IN-COMPLETE",
                    "timeStamp": timeStamp
                }
                return [], [dbTransactionRequest], []

            
            internalTransactionRequest = {
                "stockId": stockId,
                "sellerId": sellerId,
                "sellerTid": sellerTransactionId,
                "noOfStocks": totalStockSold,
                "amount": totalAmountRecieved,
                "timeStamp": timeStamp
            }

            dbTransactionRequest = {
                "tId": sellerTransactionId,
                "uId": sellerId,
                "stockId": stockId,
                "side": "sell",
                "orderType": "market",
                "quantity": totalStockSold,
                "pricePerUnit": totalAmountRecieved / totalStockSold if totalStockSold else 0,
                "status": "PARTIAL" if numberOfStockAvailable > 0 else "COMPLETED",
                "timeStamp": timeStamp
            }

            userTransactionRequest = [
                {
                    "action": "add",
                    "resource": "money",
                    "uId": sellerId,
                    "quantity": totalAmountRecieved
                }
            ]

            internalTransactions.append(internalTransactionRequest)
            dbTransactions.append(dbTransactionRequest)
            userTransactions.append(userTransactionRequest)

        return internalTransactions, dbTransactions, userTransactions
                
    def limitTransaction(request):
        nonlocal transactions, stockId
        timeStamp = time.time()
        side = request.get("side")
        internalTransactions = []   # Transactions from one user to another
        dbTransactions = [] # Database Transactions
        userTransactions = []   # User Transactions

        if side == "buy":
            buyerTransaction = request
            numberOfStocksRequired = buyerTransaction.get("quantity")
            totalStockBrought = 0
            buyerId = buyerTransaction.get("uId"); buyerTransactionId = buyerTransaction.get("tId")
            maxBuyPrice = buyerTransaction.get("pricePerUnit")
            priceToBePaidByBuyer = 0
            
            while numberOfStocksRequired > 0 and len(transactions["sell"]) > 0 and transactions["sell"][0][0] <= maxBuyPrice:
                priceOfBestSell, thisTimeStamp, bestSell = heapq.heappop(transactions["sell"]) # 
                numberOfStocksAvailableInBestSell = bestSell.get("quantity")

                stocksInThisTransaction = min(numberOfStocksAvailableInBestSell, numberOfStocksRequired)
                numberOfStocksAvailableInBestSell -= stocksInThisTransaction
                numberOfStocksRequired -= stocksInThisTransaction
                totalStockBrought += stocksInThisTransaction

                priceInThisTransaction = stocksInThisTransaction * priceOfBestSell
                priceToBePaidByBuyer += priceInThisTransaction

                transactions["marketPrice"] = priceOfBestSell

                sellerId = bestSell.get("uId")
                sellerTid = bestSell.get("tId")
                internalTransactionRequest = {
                    "stockId": stockId,
                    "sellerId": sellerId,
                    "sellerTid": sellerTid,
                    "buyerId": buyerId,
                    "buyerTid": buyerTransactionId,
                    "noOfStocks": stocksInThisTransaction,
                    "amount": priceInThisTransaction,
                    "timeStamp": timeStamp
                }

                dbTransactionRequest = {
                    "tId": sellerTid,
                    "uId": sellerId,
                    "stockId": stockId,
                    "side": "sell",
                    "orderType": "limit",
                    "quantity": stocksInThisTransaction,
                    "pricePerUnit": priceOfBestSell,
                    "status": "PARTIAL" if numberOfStocksAvailableInBestSell > 0 else "COMPLETED",
                    "timeStamp": timeStamp
                }

                userTransactionRequest = [
                        {
                            "action": "add",
                            "resource": "money",
                            "uId": sellerId,
                            "quantity": priceInThisTransaction
                        }
                    ]

                internalTransactions.append(internalTransactionRequest)
                dbTransactions.append(dbTransactionRequest)
                userTransactions.append(userTransactionRequest)

                if numberOfStocksAvailableInBestSell > 0:
                    # If there are more stocks to sell than the required, add them back to the heap
                    bestSell["quantity"] = numberOfStocksAvailableInBestSell
                    heapq.heappush(transactions["sell"], (priceOfBestSell, thisTimeStamp, bestSell))
            
            # If no stocks were brought then add the empty request
            if totalStockBrought == 0:
                heapq.heappush(transactions["buy"], (-request.get("pricePerUnit"), request.get("timeStamp"), request))
                return [], [], []
            
            if numberOfStocksRequired > 0:
                request["quantity"] = numberOfStocksRequired
                heapq.heappush(transactions["buy"], (-request.get("pricePerUnit"), request.get("timeStamp"), request))
            
            internalTransactionRequest = {
                "stockId": stockId,
                "buyerId": buyerId,
                "buyerTid": buyerTransactionId,
                "noOfStocks": totalStockBrought,
                "amount": priceToBePaidByBuyer,
                "timeStamp": timeStamp
            }

            dbTransactionRequest = {
                "tId": buyerTransactionId,
                "uId": buyerId,
                "stockId": stockId,
                "side": "buy",
                "orderType": "limit",
                "quantity": totalStockBrought,
                "pricePerUnit": priceToBePaidByBuyer / totalStockBrought if totalStockBrought else 0,
                "status": "PARTIAL" if numberOfStocksRequired > 0 else "COMPLETED",
                "timeStamp": timeStamp
            }

            userTransactionRequest = [
                {
                    "action": "add",
                    "resource": "stock",
                    "uId": buyerId,
                    "stockId": stockId,
                    "quantity": totalStockBrought
                }
            ]

            internalTransactions.append(internalTransactionRequest)
            dbTransactions.append(dbTransactionRequest)
            userTransactions.append(userTransactionRequest)

        else:
            # This is Sell Part
            sellerTransaction = request
            sellerId, sellerTransactionId = sellerTransaction.get("uId"), sellerTransaction.get("tId")
            minimumSellingPrice = sellerTransaction.get("pricePerUnit")
            totalStockSold = 0; totalAmountRecieved = 0
            numberOfStockAvailable = sellerTransaction.get("quantity")
            while numberOfStockAvailable > 0 and len(transactions["buy"]) > 0 and -transactions["buy"][0][0] >= minimumSellingPrice:
                buyPrice, thisTimeStamp, bestBuy = heapq.heappop(transactions["buy"])
                buyPrice *= -1  # Since the data is saved in negative prices for max-heap
                numberOfBestStocksToBuy = bestBuy.get("quantity")

                stocksSoldInThisTransaction = min(numberOfStockAvailable, numberOfBestStocksToBuy)
                priceInThisTransaction = buyPrice * stocksSoldInThisTransaction
                totalAmountRecieved += priceInThisTransaction

                transactions["marketPrice"] = buyPrice
                numberOfStockAvailable -= stocksSoldInThisTransaction
                numberOfBestStocksToBuy -= stocksSoldInThisTransaction
                totalStockSold += stocksSoldInThisTransaction

                buyerId = bestBuy.get("uId"); buyerTid = bestBuy.get("tId")
                internalTransactionRequest = {
                    "stockId": stockId,
                    "sellerId": sellerId,
                    "sellerTid": sellerTransactionId,
                    "buyerId": buyerId,
                    "buyerTid": buyerTid,
                    "noOfStocks": stocksSoldInThisTransaction,
                    "amount": priceInThisTransaction,
                    "timeStamp": timeStamp
                }

                dbTransactionRequest = {
                    "tId": buyerTid,
                    "uId": buyerId,
                    "stockId": stockId,
                    "side": "buy",
                    "orderType": "limit",
                    "quantity": stocksSoldInThisTransaction,
                    "pricePerUnit": buyPrice,
                    "status": "PARTIAL" if numberOfBestStocksToBuy > 0 else "COMPLETED",
                    "timeStamp": timeStamp
                }

                userTransactionRequest = [
                {
                    "action": "add",
                    "resource": "stock",
                    "uId": buyerId,
                    "stockId": stockId,
                    "quantity": stocksSoldInThisTransaction
                }
                ]

                if numberOfBestStocksToBuy > 0:
                    bestBuy["quantity"] = numberOfBestStocksToBuy
                    heapq.heappush(transactions["buy"], (-1 * buyPrice, thisTimeStamp, bestBuy))
            
            if totalStockSold == 0:
                heapq.heappush(transactions["sell"], (request.get("pricePerUnit"), request.get("timeStamp"), request))
                return [], [], []

            if numberOfStockAvailable > 0:
                request["quantity"] = numberOfStockAvailable
                heapq.heappush(transactions["sell"], (request.get("pricePerUnit"), request.get("timeStamp"), request))

            internalTransactionRequest = {
                "stockId": stockId,
                "sellerId": sellerId,
                "sellerTid": sellerTransactionId,
                "noOfStocks": totalStockSold,
                "amount": totalAmountRecieved,
                "timeStamp": timeStamp
            }

            dbTransactionRequest = {
                "tId": sellerTransactionId,
                "uId": sellerId,
                "stockId": stockId,
                "side": "sell",
                "orderType": "limit",
                "quantity": totalStockSold,
                "pricePerUnit": totalAmountRecieved / totalStockSold if totalStockSold else 0,
                "status": "PARTIAL" if numberOfStockAvailable > 0 else "COMPLETED",
                "timeStamp": timeStamp
            }

            userTransactionRequest = [
                {
                    "action": "add",
                    "resource": "money",
                    "uId": sellerId,
                    "quantity": totalAmountRecieved
                }
            ]

            internalTransactions.append(internalTransactionRequest)
            dbTransactions.append(dbTransactionRequest)
            userTransactions.append(userTransactionRequest)

        return internalTransactions, dbTransactions, userTransactions
        

    try:
        transactions = {"buy":[], "sell": [], "marketPrice": 0.0}
        while True:
            if shutdownEvent.is_set():
                break
            request = None
            try:
                request = queue.get(timeout=0.01)
            except QueueEmpty as qe:
                print("No Transaction recieved, Lets wait!")
                continue
            
            if request is None:
                continue
            orderType = request.get("orderType")
            internalTxns, dbTxns, userTxns =[], [], []
            if orderType == "limit":
                internalTxns, dbTxns, userTxns = limitTransaction(request)
            elif orderType == "market":
                internalTxns, dbTxns, userTxns = marketTransaction(request)
            elif orderType == "ioc":
                pass
            elif orderType == "pok":
                pass
            else:
                # Un-known order type
                print("Recieved Unknown order type: ", request)
            

            if dbTxns:
                for txn in dbTxns:
                    dbQueue.put(txn)
            
            if internalTxns:
                for txn in internalTxns:
                    iTQueue.put(txn)
            
            if userTxns:
                for txn in userTxns:
                    uId = txn.get("uId")
                    userData = users[uId]
                    resource = txn.get("resourcse")
                    if resource == "stock":
                        action = txn.get("action")
                        stockId = txn.get("stockId")
                        quantity = txn.get("quantity")
                        if action == "add":    
                            if stockId in userData["stocks"]:
                                userData["stocks"][stockId] += quantity
                            else:
                                userData["stocks"][stockId] = quantity
                        else:
                            if stockId in userData:
                                userData["stocks"][stockId] -= quantity
                            else:
                                userData["stocks"][stockId] = -quantity
                    
                    else:
                        action = txn.get("action")
                        amount = txn.get("quantity")
                        if action == "add":
                            userData["walletBalance"] += amount
                        else:
                            userData["walletBalance"] -= amount

                    users[uId] = userData
                    

    except Exception as _e:
        logQueue.put(f"Exception at transaction-engine {stockId}: {str(_e)}")
    
    finally:
        print("Exiting")


In [ ]:
import heapq

def marketTransaction(request):
    nonlocal transactions, stockId
    side = request.get("side")
    internalTransactions = []; dbTransactions = []; userTransactions = []
    timeStamp = time.time()

    if side == "buy":
        buyerId, buyerTid = request.get("uId"), request.get("tId")
        numberOfStocksRequired = totalNumberOfStocksRequired = request.get("quantity"); amountPaid = request.get("pricePerUnit") * numberOfStocksRequired
        totalAmountRequired = 0; totalStocksBrought = 0

        while numberOfStocksRequired > 0 and len(transactions["sell"]) > 0:
            sellingPrice, sellingTimeStamp, sellRequest = heapq.heappop(transactions["sell"])
            stocksAvaliableInThisTransaction = sellRequest.get("quantity")

            stocksInThisTransaction = min(numberOfStocksRequired, stocksAvaliableInThisTransaction)
            numberOfStocksRequired -= stocksInThisTransaction
            stocksAvaliableInThisTransaction -= stocksInThisTransaction
            priceOfThisTransaction = stocksInThisTransaction * sellingPrice
            totalAmountRequired += priceOfThisTransaction
            totalStocksBrought += stocksInThisTransaction

            if stocksAvaliableInThisTransaction > 0:
                # Update this back to the heap
                sellRequest["quantity"] = stocksAvaliableInThisTransaction
                heapq.heappush(transactions["sell"], (sellingPrice, sellingTimeStamp, sellRequest))

                sellerId, sellerTid = sellRequest.get("uId"), sellRequest.get("tId")
                internalTransactionRequest = {
                    "stockId": stockId,
                    "sellerId": sellerId,
                    "sellerTid": sellerTid,
                    "buyerId": buyerId,
                    "buyerTid": buyerTid,
                    "noOfStocks": stocksInThisTransaction,
                    "amount": priceOfThisTransaction,
                    "timeStamp": timeStamp
                }

                dbTransactionRequest = {
                    "tId": sellerTid,
                    "uId": sellerId,
                    "stockId": stockId,
                    "side": "sell",
                    "orderType": "market",
                    "quantity": stocksInThisTransaction,
                    "pricePerUnit": sellingPrice,
                    "status": "PARTIAL" if stocksAvaliableInThisTransaction > 0 else "COMPLETED",
                    "timeStamp": timeStamp
                }

                userTransactionRequest = [
                        {
                            "action": "add",
                            "resource": "money",
                            "uId": sellerId,
                            "quantity": priceOfThisTransaction
                        }
                    ]

                internalTransactions.append(internalTransactionRequest)
                dbTransactions.append(dbTransactionRequest)
                userTransactions.append(userTransactionRequest)

        if totalStocksBrought == 0:
            dbTransactionRequest = {
                    "tId": buyerTid,
                    "uId": buyerId,
                    "stockId": stockId,
                    "side": "buy",
                    "orderType": "market",
                    "quantity": numberOfStocksRequired,
                    "pricePerUnit": request.get("pricePerUnit"),
                    "status": "IN-COMPLETE",
                    "timeStamp": timeStamp
                }
            userTransactionRequest = {
                "action": "add",
                "resource": "money",
                "quantity": amountPaid, # Refund the total amount to the user
                "uId": buyerId
            }

            return [], [dbTransactionRequest], [userTransactionRequest]
        
        dbTransactionRequest = {
            "tId": buyerTid,
            "uId": buyerId,
            "stockId": stockId,
            "side": "buy",
            "orderType": "market",
            "quantity": totalStocksBrought,
            "pricePerUnit": totalAmountRequired / totalStocksBrought if totalStocksBrought > 0 else 0,
            "status": "PARTIAL" if numberOfStocksRequired > 0 else "COMPLETED",
            "timeStamp": timeStamp
        }

        userTransactionRequest = {
            "action": "add",
            "resource": "stock",
            "stockId": stockId,
            "quantity": totalStocksBrought,
            "uId": buyerId
        }
        
        dbTransactions.append(dbTransactionRequest)
        userTransactions.append(userTransactionRequest)
        amountDifference = amountPaid - totalAmountRequired
        if amountDifference != 0:
            userTransactionRequest = {
                "action": "add" if amountPaid > totalAmountRequired else "remove",
                "resource": "money",
                "uId": buyerId,
                "quantity": abs(amountDifference)
            }

            userTransactions.append(userTransactionRequest)
        
        return internalTransactions, dbTransactions, userTransactions
    
    else:
        sellerId, sellerTid = request.get("uId"), request.get("tId")
        numberOfStocksToSell = totalNumberOfStocksToSell = request.get("quantity")
        totalStocksSold = totalAmountRecieved = 0

        while numberOfStocksToSell > 0 and len(transactions["buy"]) > 0:
            buyingPrice, buyingTimeStamp, buyRequest = heapq.heappop(transactions["buy"])
            buyingPrice *= -1
            numberOfStocksRequired = buyRequest.get("quantity")

            stocksInThisTransaction = min(numberOfStocksRequired, numberOfStocksToSell)
            numberOfStocksRequired -= stocksInThisTransaction
            numberOfStocksToSell -= stocksInThisTransaction

            priceOfThisTransaction = stocksInThisTransaction * buyingPrice
            totalStocksSold += stocksInThisTransaction
            totalAmountRecieved += priceOfThisTransaction

            if numberOfStocksRequired > 0:
                # Push the transaction back to the heap
                buyRequest["quantity"] = numberOfStocksRequired
                heapq.heappush(transactions["buy"], (-buyingPrice, buyingTimeStamp, buyRequest))
            
            buyerId, buyerTid = buyRequest.get("uId"), buyRequest.get("tId")
            internalTransactionRequest = {
                "stockId": stockId,
                "sellerId": sellerId,
                "sellerTid": sellerTid,
                "buyerId": buyerId,
                "buyerTid": buyerTid,
                "noOfStocks": stocksInThisTransaction,
                "amount": priceOfThisTransaction,
                "timeStamp": timeStamp
            }

            dbTransactionRequest = {
                "tId": sellerTid,
                "uId": sellerId,
                "stockId": stockId,
                "side": "buy",
                "orderType": "market",
                "quantity": stocksInThisTransaction,
                "pricePerUnit": buyingPrice,
                "status": "PARTIAL" if numberOfStocksRequired > 0 else "COMPLETED",
                "timeStamp": timeStamp
            }

            userTransactionRequest = {
                "action": "add",
                "resource": "stock",
                "stockId": stockId,
                "quantity": stocksInThisTransaction,
                "uId": buyerId
            }

            internalTransactions.append(internalTransactionRequest)
            dbTransactions.append(dbTransactionRequest)
            userTransactions.append(userTransactionRequest)
        
        if totalStocksSold == 0:
            userTransactionRequest = {
                "action": "add",
                "resource": "stock",
                "stockId": stockId,
                "uId": sellerId,
                "quantity": totalNumberOfStocksToSell
            }

            dbTransactionRequest = {
                "tId": sellerTid,
                "uId": sellerId,
                "stockId": stockId,
                "side": "sell",
                "orderType": "market",
                "quantity": totalNumberOfStocksToSell,
                "pricePerUnit": request.get("pricePerUnit"),
                "status": "IN-COMPLETE",
                "timeStamp": timeStamp
            }

            return [], [dbTransactions], [userTransactionRequest]

        dbTransactionRequest = {
            "tId": sellerTid,
            "uId": sellerId,
            "stockId": stockId,
            "side": "sell",
            "orderType": "market",
            "quantity": totalStocksSold,
            "pricePerUnit": totalAmountRecieved / totalStocksSold if totalStocksSold else 0,
            "status": "PARTIAL" if numberOfStocksToSell > 0 else "COMPLETED",
            "timeStamp": timeStamp
        }
        dbTransactions.append(dbTransactionRequest)

        userTransactionRequest = {
            "action": "add",
            "resource": "money",
            "uId": sellerId,
            "quantity": totalAmountRecieved
        }

        userTransactions.append(userTransactionRequest)

        if numberOfStocksToSell > 0:
            userTransactionRequest = {
                "action": "add",
                "resource": "stock",
                "stockId": stockId,
                "uId": sellerId,
                "quantity": numberOfStocksToSell
            }
            userTransactions.append(userTransactionRequest)
        return internalTransactions, dbTransactions, userTransactions

transactions = {"buy": [], "sell": [], "marketPrice": 0.0}
stockId = "btc"

In [ ]:
# Check for balance
# CHeck for the transfer of stock